In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from urllib.request import urlopen

In [3]:
requestHead = {"User-Agent": "Chrome/47.0.2526.111"}

***
testing

In [16]:
strnum = '02'
url = "http://sim-football.com/indexes/NSFLS"+strnum+"/LeagueDefenseStats.html"
page = requests.get(url, headers = requestHead)
soup = BeautifulSoup(page.text,'html.parser')

In [18]:
soup.find_all('table',class_ = 'tablesorter')

[]

In [17]:
kickTable = soup.find_all('table', class_ = 'tablesorter')[0]

IndexError: list index out of range

In [10]:
kickCols=['Name','Pos','Team','G','Punts','Yds','Avg','Inside 20','Long']

In [11]:
testList = []
for row in kickTable.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) > 0:
        testList.append([t.text for t in cols])
kickDF = pd.DataFrame(testList,columns=[t.text for t in kickTable.find_all('tr')[0].find_all('th')])

In [14]:
for i in kickDF.columns:
    try:
        kickDF[i] = pd.to_numeric(kickDF[i])
    except ValueError:
        continue

In [174]:

kickDF['teamID'] = kickDF.apply(lambda row: teamID(row,int(strnum)),axis=1)

In [15]:
kickDF

,Name,Pos,Team,G,Punts,Yds,Avg,Inside 20,Long
0,"Harris, S.",K,PHI,14,120,5586,46.6,25,66
1,"Kicksit, B.",K,COL,14,119,5376,45.2,19,71
2,"Ronaldo, C.",P,ARI,14,118,5451,46.2,17,71
3,"Blewitt, I.",K,LV,14,111,5027,45.3,20,63
4,"Turkleton, T.",K,BAL,14,103,4529,44.0,16,67
5,"Booter, A.",K,YKW,14,87,3867,44.4,13,65
6,"Langdon, J.",K,OCO,14,75,3402,45.4,14,65
7,"Nugent, A.",K,SJS,6,56,2318,41.4,10,55
8,"Laces, K.",K,SJS,8,48,2068,43.1,4,63
9,"Sims, A.",K,OCO,3,22,974,44.3,4,60


In [176]:
for dist in ['<20','20-29','30-39','40-49','50+']:
    kickDF[dist] = kickDF.apply(lambda row: percentDist(row),axis=1)

In [177]:
kickDF

,Name,Pos,Team,G,XPM,XPA,XP%,FGM,FGA,FG%,<20,20-29,30-39,40-49,50+,Lng,teamID
0,"Ronaldo, C.",P,ARI,14,40,41,97.6,25,29,86.2,0.0,100.000000,100.000000,66.666667,0.0,46,4
1,"Turkleton, T.",K,BAL,14,23,24,95.8,33,37,89.2,100.0,90.909091,92.307692,77.777778,100.0,57,1
2,"Booter, A.",K,YKW,14,28,31,90.3,28,30,93.3,100.0,100.000000,100.000000,91.666667,50.0,57,2
3,"Kicksit, B.",K,COL,14,28,31,90.3,18,22,81.8,100.0,85.714286,85.714286,66.666667,50.0,57,3
4,"Harris, S.",K,PHI,14,25,25,100.0,20,26,76.9,100.0,100.000000,66.666667,60.000000,80.0,57,7
5,"Langdon, J.",K,OCO,14,19,19,100.0,21,29,72.4,100.0,100.000000,71.428571,58.333333,0.0,47,5
6,"Blewitt, I.",K,LV,14,21,23,91.3,14,22,63.6,100.0,50.000000,57.142857,70.000000,0.0,47,8
7,"Laces, K.",K,SJS,8,17,18,94.4,14,18,77.8,100.0,100.000000,66.666667,100.000000,0.0,46,6
8,"Nugent, A.",K,SJS,6,6,7,85.7,6,9,66.7,100.0,100.000000,100.000000,40.000000,0.0,47,6
9,"Sims, A.",K,OCO,3,9,9,100.0,2,3,66.7,0.0,0.000000,50.000000,100.000000,0.0,43,5


In [178]:
kickerDF = pd.DataFrame(columns = ['Ovr', 'Str', 'Agi', 'Arm', 'Int', 'Acc', 'Tck', 'Spe','Hnd', 'PBl', 'RBl', 'KDi', 'KAc', 'End'])

In [179]:
for p in range(len(kickDF)):
    print(p)
    playerRow = kickDF.loc[p]
    playerName = playerRow['Name']
    playerTeam = playerRow['teamID']
    
    urlTeam = 'http://sim-football.com/indexes/NSFLS%s/Teams/%s.html'%(strnum,str(playerTeam))
    pageTeam = requests.get(urlTeam, headers = requestHead)
    soupTeam = BeautifulSoup(pageTeam.text,'html.parser')
    rosterSoup = soupTeam.find_all(id = 'tabs-2')[0]
    rosterTable = rosterSoup.find_all('table',class_='Grid')[0]
    
    playerList = []
    for row in rosterTable.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 0:
            playerList.append([t.text for t in cols])
    teamDF = pd.DataFrame(playerList)
    teamDF.columns = teamDF.iloc[0]
    teamDF = teamDF[1:]
    try:
        playerDF = teamDF[teamDF['Player'] == playerName][['Player','Ovr', 'Str', 'Agi', 'Arm', 'Int', 'Acc', 'Tck', 'Spe','Hnd', 'PBl', 'RBl', 'KDi', 'KAc', 'End']]
        playerDF.index = [p]
        kickerDF = pd.concat([kickerDF,playerDF])
    except ValueError:
        kickDF = kickDF.drop([p],axis=0)

0


/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



1
2
3
4
5
6
7
8
9


In [180]:
kickerDF

,Acc,Agi,Arm,End,Hnd,Int,KAc,KDi,Ovr,PBl,Player,RBl,Spe,Str,Tck
0,1,30,20,67,15,58,81,90,76,1,"Ronaldo, C.",1,30,25,15
1,1,30,20,68,15,67,89,81,78,1,"Turkleton, T.",1,30,25,15
2,1,32,20,71,15,72,80,82,77,1,"Booter, A.",1,35,27,17
3,1,30,20,65,15,60,80,85,73,1,"Kicksit, B.",1,30,25,15
4,1,35,20,70,15,72,85,85,81,1,"Harris, S.",1,50,25,25
5,1,35,20,65,15,55,75,75,68,1,"Langdon, J.",1,40,20,15
6,1,36,20,67,15,71,80,77,74,1,"Blewitt, I.",1,37,15,15
8,1,30,20,70,15,60,78,65,67,1,"Nugent, A.",1,30,15,15
9,1,35,20,73,15,55,65,70,64,1,"Sims, A.",1,40,45,15


In [181]:
kickDF = pd.concat([kickDF,kickerDF],axis=1)

In [182]:
kickDF = kickDF.sort_values('teamID').set_index('teamID')

In [183]:
kickDF

,Name,Pos,Team,G,XPM,XPA,XP%,FGM,FGA,FG%,...,Int,KAc,KDi,Ovr,PBl,Player,RBl,Spe,Str,Tck
teamID,,,,,,,,,,,,,,,,,,,,,
1,"Turkleton, T.",K,BAL,14,23,24,95.8,33,37,89.2,...,67,89,81,78,1,"Turkleton, T.",1,30,25,15
2,"Booter, A.",K,YKW,14,28,31,90.3,28,30,93.3,...,72,80,82,77,1,"Booter, A.",1,35,27,17
3,"Kicksit, B.",K,COL,14,28,31,90.3,18,22,81.8,...,60,80,85,73,1,"Kicksit, B.",1,30,25,15
4,"Ronaldo, C.",P,ARI,14,40,41,97.6,25,29,86.2,...,58,81,90,76,1,"Ronaldo, C.",1,30,25,15
5,"Langdon, J.",K,OCO,14,19,19,100.0,21,29,72.4,...,55,75,75,68,1,"Langdon, J.",1,40,20,15
5,"Sims, A.",K,OCO,3,9,9,100.0,2,3,66.7,...,55,65,70,64,1,"Sims, A.",1,40,45,15
6,"Nugent, A.",K,SJS,6,6,7,85.7,6,9,66.7,...,60,78,65,67,1,"Nugent, A.",1,30,15,15
7,"Harris, S.",K,PHI,14,25,25,100.0,20,26,76.9,...,72,85,85,81,1,"Harris, S.",1,50,25,25
8,"Blewitt, I.",K,LV,14,21,23,91.3,14,22,63.6,...,71,80,77,74,1,"Blewitt, I.",1,37,15,15


In [184]:
urlStand = 'http://sim-football.com/indexes/NSFLS%s/Index.html'%strnum
pageStand = requests.get(urlStand,headers=requestHead)
soupStand = BeautifulSoup(pageStand.text,'html.parser')


In [185]:
standList = soupStand.find_all('table', class_='Grid')
NSFCstand = standList[0]
ASFCstand = standList[1]

In [186]:
teamList = []
for stand in [NSFCstand,ASFCstand]:
    for row in stand.find_all('tr'):
        cols = row.find_all('td')
        links = row.find_all('a')
        if len(links) > 0:
#             print(links)
#             print(cols)
            teamList.append([t.text for t in cols])
standDF = pd.DataFrame(teamList,columns=['Team','W','L','T','Pct','PF','PA','Home','Road','Conf','Div','Str'])

In [187]:
for i in ['W','L','T','Pct','PF','PA']:
    standDF[i] = pd.to_numeric(standDF[i])

In [188]:
standDF['teamID'] = standDF.apply(lambda row: standID(int(strnum),row),axis=1)

In [189]:
standDF = standDF.sort_values('teamID').set_index('teamID')

In [190]:
standDF = standDF[['W','L','T','Pct','PF','PA']]

In [191]:
standDF

,W,L,T,Pct,PF,PA
teamID,,,,,,
1,8,6,0,0.571,268,270
2,8,6,0,0.571,300,295
3,8,6,0,0.571,268,216
4,12,2,0,0.857,369,175
5,8,6,0,0.571,271,219
6,4,10,0,0.286,233,374
7,5,9,0,0.357,245,281
8,3,11,0,0.214,211,335


In [192]:
kickDF = pd.concat([kickDF,standDF],axis=1)

In [193]:
kickDF['S'] = int(strnum)

In [194]:
kickDF

,Name,Pos,Team,G,XPM,XPA,XP%,FGM,FGA,FG%,...,Spe,Str,Tck,W,L,T,Pct,PF,PA,S
teamID,,,,,,,,,,,,,,,,,,,,,
1,"Turkleton, T.",K,BAL,14,23,24,95.8,33,37,89.2,...,30,25,15,8,6,0,0.571,268,270,2
2,"Booter, A.",K,YKW,14,28,31,90.3,28,30,93.3,...,35,27,17,8,6,0,0.571,300,295,2
3,"Kicksit, B.",K,COL,14,28,31,90.3,18,22,81.8,...,30,25,15,8,6,0,0.571,268,216,2
4,"Ronaldo, C.",P,ARI,14,40,41,97.6,25,29,86.2,...,30,25,15,12,2,0,0.857,369,175,2
5,"Langdon, J.",K,OCO,14,19,19,100.0,21,29,72.4,...,40,20,15,8,6,0,0.571,271,219,2
5,"Sims, A.",K,OCO,3,9,9,100.0,2,3,66.7,...,40,45,15,8,6,0,0.571,271,219,2
6,"Nugent, A.",K,SJS,6,6,7,85.7,6,9,66.7,...,30,15,15,4,10,0,0.286,233,374,2
7,"Harris, S.",K,PHI,14,25,25,100.0,20,26,76.9,...,50,25,25,5,9,0,0.357,245,281,2
8,"Blewitt, I.",K,LV,14,21,23,91.3,14,22,63.6,...,37,15,15,3,11,0,0.214,211,335,2
